In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 0) util: returns + features vol
def add_returns(df):
    # 1) log ret
    df = df.copy()
    df["ret"] = np.log(df["close"] / df["close"].shift(1))
    return df

def add_vol_features(df, lam=0.94, win=20):
    # 2) ewma
    df["vol_ewma"] = np.sqrt((df["ret"] ** 2).ewm(alpha=1 - lam).mean())

    # 3) rolling
    df["vol_roll20"] = df["ret"].rolling(win).std()

    # 4) zscore regime (252j)
    m = df["vol_ewma"].rolling(252).mean()
    s = df["vol_ewma"].rolling(252).std()
    df["vol_z"] = (df["vol_ewma"] - m) / s
    return df


# 1) util: backtest
def run_backtest(weights, rets, cost_bps=10):
    # 1a) lag
    w = weights.shift(1).fillna(0.0)

    # 1b) long-only
    w = w.clip(lower=0)

    # 1c) normalisation
    denom = w.sum(axis=1).replace(0, np.nan)
    w = w.div(denom, axis=0).fillna(0.0)

    # 1d) port ret
    port_gross = (w * rets).sum(axis=1)

    # 1e) turnover + costs
    turnover = w.diff().abs().sum(axis=1).fillna(0.0)
    costs = turnover * (cost_bps / 10000.0)

    # 1f) net
    port_net = port_gross - costs
    return port_net, turnover


# 2) util: perf
def max_drawdown(equity):
    peak = equity.cummax()
    dd = equity / peak - 1.0
    return float(dd.min())

def perf_stats(port_ret, freq=365):
    port_ret = port_ret.dropna()

    # NOTE: si ret = log, equity plus correct = exp(cumsum)
    equity = np.exp(port_ret.cumsum())

    mu = port_ret.mean() * freq
    sig = port_ret.std() * np.sqrt(freq)
    sharpe = float(mu / sig) if sig > 0 else np.nan
    mdd = max_drawdown(equity)

    # ann return geom
    ann_ret = float(equity.iloc[-1] ** (freq / len(port_ret)) - 1) if len(port_ret) > 0 else np.nan

    return {
        "ann_return": ann_ret,
        "ann_vol": float(sig),
        "sharpe": sharpe,
        "max_dd": mdd,
    }


# 3) build panel depuis all_data (pas de load_symbol)
# 3a) symbols dispo
SYMBOLS_BT = list(all_data.keys())  # prend exactement ceux du benchmark

dfs = {}
for s in SYMBOLS_BT:
    df = all_data[s].copy()
    df = add_returns(df)
    df = add_vol_features(df)
    dfs[s] = df.dropna(subset=["ret"])

# 3b) dates communes
common_idx = None
for s, df in dfs.items():
    common_idx = df.index if common_idx is None else common_idx.intersection(df.index)

# 3c) returns matrix
rets = pd.DataFrame({s: dfs[s].loc[common_idx, "ret"] for s in SYMBOLS_BT}).dropna()

# 3d) vols matrix
vol  = pd.DataFrame({s: dfs[s].loc[rets.index, "vol_ewma"] for s in SYMBOLS_BT})
volz = pd.DataFrame({s: dfs[s].loc[rets.index, "vol_z"] for s in SYMBOLS_BT})

# 4) strat 0: equal weight
w_bh = pd.DataFrame(1.0, index=rets.index, columns=rets.columns)

# 5) strat 1: inverse vol
w_invvol = 1.0 / vol.replace(0, np.nan)
w_invvol = w_invvol.fillna(0.0)

# 6) strat 2: low vol (median)
q = vol.quantile(0.5, axis=1)
w_lowvol = vol.le(q, axis=0).astype(float)

# 7) strat 3: regime (risk-on/off)
avg_z = volz.mean(axis=1)
risk_on = (avg_z < 1.0).astype(float)
w_regime = w_invvol.mul(risk_on, axis=0)

# 8) run backtests
bt = {}
bt["buy_hold_eq"] = run_backtest(w_bh, rets, cost_bps=0)
bt["inv_vol"]     = run_backtest(w_invvol, rets, cost_bps=10)
bt["low_vol"]     = run_backtest(w_lowvol, rets, cost_bps=10)
bt["regime_iv"]   = run_backtest(w_regime, rets, cost_bps=10)

# 9) summary
rows = []
for name, (r, to) in bt.items():
    st = perf_stats(r, freq=365)
    st["turnover_mean"] = float(to.mean())
    st["turnover_med"]  = float(to.median())
    st["name"] = name
    rows.append(st)

summary = pd.DataFrame(rows).set_index("name").sort_values("sharpe", ascending=False)
print(summary)

# 10) plot equity
plt.figure(figsize=(12, 5))
for name, (r, _) in bt.items():
    eq = np.exp(r.fillna(0).cumsum())  # log-ret -> equity correct
    plt.plot(eq.index, eq, label=name)

plt.legend()
plt.title("Naive strategies equity (net)")
plt.tight_layout()
plt.show()


              ann_return   ann_vol    sharpe    max_dd  turnover_mean  \
name                                                                   
regime_iv     -0.046037  0.538442 -0.087531 -0.373991       0.038552   
low_vol       -0.161650  0.498715 -0.353548 -0.390482       0.025506   
inv_vol       -0.243114  0.583303 -0.477527 -0.459000       0.019322   
buy_hold_eq   -0.302622  0.693492 -0.519728 -0.559583       0.002639   

             turnover_med  
name                       
regime_iv        0.010051  
low_vol          0.000000  
inv_vol          0.010846  
buy_hold_eq      0.000000  


![Image test3](image/image_test_3.png)


Les résultats montrent que la performance des estimateurs de volatilité dépend fortement de l’horizon de prévision et de l’actif considéré. À court horizon (5 jours), les modèles ARCH/GARCH apportent souvent une amélioration par rapport aux estimateurs simples, en particulier pour les cryptomonnaies les plus liquides (BTC, BNB, SOL, XRP). 
Les spécifications EGARCH et les distributions Student-t se révèlent parfois plus adaptées, ce qui est cohérent avec la présence de rendements asymétriques et de valeurs extrêmes sur les marchés crypto.

À horizon plus long (20 jours), les méthodes simples comme la volatilité rolling ou l’EWMA dominent largement. Cela s’explique par le fait que la volatilité réalisée sur 20 jours est déjà fortement lissée, ce qui favorise naturellement des estimateurs basés sur un lissage similaire. Dans ce contexte, la complexité supplémentaire des modèles GARCH n’apporte pas de gain significatif.

Enfin, plusieurs avertissements de convergence apparaissent lors de l’estimation des modèles GARCH, ce qui est courant sur des données financières volatiles et dans un cadre de réestimation fréquente. Ces limites sont prises en compte dans l’interprétation et ouvrent la voie à des améliorations méthodologiques dans les étapes suivantes du projet.